### Importing some pre-trained networks
My computer is not powerful enough to handle the convolutional network 

In [ ]:
from torchvision import models
import torch

resnet = torch.hub.load('NVIDIA/DeepLearningExamples:torchhub', 'nvidia_resnet50', pretrained=True)
print(resnet)

vgg16 = models.vgg16(pretrained=True)
print(vgg16)

resnet34 = models.resnet34(pretrained=True)
print(resnet34)

## ended up going with vgg16 because it has less layers and there are papers 
## transfering the convolutional layers from vgg16 and re-training the classification
## layers with great success

### Keeping the convolutional layers
Makes the convolutional layers in the vgg16 model grad-less so backpropogating doesn't change its weights. Prints the layers in the network that require a grad to verify

In [ ]:
for param in vgg16.features.parameters():
    param.requires_grad = False

for name, param in vgg16.named_parameters():
    if param.requires_grad == True:
        print(name)

### New calssification layers
Inserts new classification layers in the place of the original ones

In [ ]:
from collections import OrderedDict

classifier = nn.Sequential(OrderedDict([
    ('fc1', nn.Linear(25088, 4096)), ('relu', nn.ReLU()),
    ('fc2', nn.Linear(4096, 1024)), ('relu', nn.ReLU()),
    ('fc3', nn.Linear(1024, 50)), ('output', nn.LogSoftmax(dim=1))
]))

vgg16.classifier = classifier
print(vgg16)


### Trains the netowrk for detecting Landmarks

In [ ]:
import gc

criterion = nn.NLLLoss()
optimizer = optim.SGD(vgg16.classifier.parameters(), lr=0.0001)
epochs = 5

#clears cache
torch.cuda.empty_cache()
#sets processing to gpu
vgg16.cuda()
#validation loss min for saving model
valid_loss_min = 10000


for e in range(epochs):
    #tracking loss
    train_loss = 0.0
    valid_loss = 0.0
    
    #sets to training mode
    vgg16.train()
    for images, labels in train_loader:
        #clears optimizer and moves data to cuda
        optimizer.zero_grad()
        images, labels = images.cuda(), labels.cuda()
        
        #runs through model
        outputs = vgg16(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        #updates loss
        train_loss += loss.item()*images.size(0)
        
        #clears cache
        del outputs
        gc.collect()
        torch.cuda.empty_cache()
    
    else:
        #sets to evaluation mode
        vgg16.eval()
        for images, labels in valid_loader:
            #switches to gpu
            images, labels = images.cuda(), labels.cuda()
        
            #runs the validation set
            with torch.no_grad():
                outputs = vgg16(images)
                loss = criterion(outputs, labels)
        
            #updates loss
            valid_loss += loss.item()*images.size(0)
            
            #clears cache
            del outputs
            gc.collect()
            torch.cuda.empty_cache()
            
    #updates overall losses
    training_loss = train_loss/len(train_loader.dataset)
    validation_loss = valid_loss/len(valid_loader.dataset)
    
    #prints progress
    print("Epoch: {} \tTraining Loss: {:.4f} \tValidation Loss: {:.4f}".format(
        e+1, training_loss, validation_loss))
    
    #updates model
    if valid_loss <= valid_loss_min:
        torch.save(vgg16.state_dict(), 'landmark_detection_vgg16.pt')
        valid_loss_min = valid_loss
        print('Saving Model...')
    
print('Done!')

### Testing loss

In [ ]:
vgg16.eval()
vgg16.cpu()
test_loss = 0
for images, labels in test_loader:

    with torch.no_grad():
        output = vgg16(images)
        loss = criterion(output, labels)
        test_loss += loss.item()*images.size(0)
        
    _ , guess_tensor = torch.topk(output, 1)
    guess = np.squeeze(guess_tensor.cpu().numpy())

    testing_loss = test_loss/len(test_loader.dataset)
    
    images = images.cpu().numpy()
    fig = plt.figure(figsize=(25, 5))
    for idx in np.arange(5):
        ax = fig.add_subplot(2, int(20/2), idx+1, xticks=[], yticks=[])
        plt.imshow(np.transpose(images[idx]))
        ax.set_title(str(classes[guess[idx]])+"\n"+str(classes[labels[idx]]))
print(f"Testing Loss: {testing_loss}")

### Location finder
Creates a library of locations and landmarks and returns the location of each landmark

In [ ]:
locations = {'Haleakala_National_Park': 'Haleakala National Park, Hawaii', 'Mount_Rainier_National_Park':'Mount Rainier National Park, Washington', 
             'Ljubljana_Castle': 'Ljubljana Castle, Slovenia', 'Dead_Sea':'Dead Sea, Jordan Rift Valley', 'Wroclaws_Dwarves': 'Dwarves of Wroclaws, Wroclaw', 
             'London_Olympic_Stadium': 'London Olympic Stadium, London', 'Niagara_Falls': 'Niagara Falls, New York', 'Stonehenge':'Stonehenges, Salisbury Plain', 
             'Grand_Canyon': 'Grand Canyon, Arizona', 'Golden_Gate_Bridge':'Golden Gate, San Fransisco', 'Edinburgh_Castle': 'Edinburgh, Scotland', 
             'Mount_Rushmore_National_Memorial': 'Mount Rushmore, South Dakota', 'Kantanagar_Temple':'Katanagar Temple, Sri Lanka', 
             'Yellowstone_National_Park':'Yellowstone National Park, Wyoming', 'Terminal_Tower':'Cleveland, Ohio', 'Central_Park': 'Central Park, New York', 
             'Eiffel_Tower': 'Eiffel Tower, Paris', 'Changdeokgung': 'Seoul, South Korea', 'Delicate_Arch': 'Delicate Arch, Utah', 'Vienna_City_Hall': 'Vienna City Hall, Vienna', 
             'Matterhorn':'Switzerland', 'Taj_Mahal':'Taj Mahal, Agra', 'Moscow_Raceway': 'Moscow Raceway, Moscow', 'Externsteine':'Meinburg, Germany', 'Soreq_Cave': 'Bet Shemesh, Israel', 
             'Banff_National_Park':'Banff National Park, Alberta', 'Pont_du_Gard':'Pont du Gard, France', 'Seattle_Japanese_Garden':'Seattle Japanese Park, Seattle', 
             'Sydney_Harbour_Bridge':'Sydney Harbour Bridge, Sydney', 'Petronas_Towers':'Kuala Lampur, Malaysia', 'Brooklyn_Bridge':'Brooklyn, New York', 
             'Washington_Monument': 'Washington Monument, Washington', 'Hanging_Temple':'Hanging Temple, China', 'Sydney_Opera_House': 'Sydney Australia', 
             'Great_Barrier_Reef':'Great Barrier Reef, Australia', 'Monumento_a_la_Revolucion':'Plaza de la Republica, Mexico City', 'Badlands_National_Park':'Badlands National Park, South Dakota', 
             'Atomium':'Atomium, Belgium', 'Forth_Bridge': 'Forth Bridge, Firth of Forth', 'Gateway_of_India':'Gateway of India, Mumbai', 'Stockholm_City_Hall':'Stockholm, Sweden', 
             'Machu_Picchu':'Machu Pichu, Peru', 'Death_Valley_National_Park':'Death Valley, California', 'Gullfoss_Falls':'Gullfoss Falls, Iceland', 
             'Trevi_Fountain':'Trevi Fountain, Rome', 'Temple_of_Heaven':'Temple of Heaven, Dongcheng', 'Great_Wall_of_China':'Great Wall of China, China', 
             'Prague_Astronomical_Clock':'Prague, Chezch Republic', 'Whitby_Abbey':'Whitby Abbey, Yorkshire', 'Temple_of_Olympian_Zeus': 'Temple of Olympian Zeus, Athens'}

def location_finder(x):
    return locations[classes[x]]

### Impliments the location finder on a batch of data

In [ ]:
dataiter = iter(valid_loader)
images, labels = dataiter.next()

with torch.no_grad():
        output = vgg16(images)
        
_ , guess_tensor = torch.topk(output, 1)
guess = np.squeeze(guess_tensor.cpu().numpy())
location_finder(guess)